# 使用思考鏈（CoT）進行推論的程式範例

以下是環境的準備， 執行前請先在 `.env` 檔案內填入您自己的 openai api key。

In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 #model="gpt-4", 
                                 temperature=0, max_tokens=2700):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## 思考鏈推論

In [3]:
delimiter = "####"
system_message = f"""
按照以下步驟回答客戶的問題。
客戶的查詢將以四個井字號作為分隔符號，\
也就是 {delimiter}。. 

步驟1：{delimiter} 首先判斷用戶是否在問關於特定產品或多種產品的問題。 \
不考慮產品類別。

步驟2：{delimiter} 如果用戶在問關於特定產品的問題， \
辨認該產品是否在以下列表中。

所有產品：
1. 產品：TechPro Ultrabook
類別：電腦與筆記型電腦
品牌：TechPro
型號：TP-UB100
保固：1年
評價：4.5
特性：13.3吋顯示器，8GB 記憶體，256GB SSD，Intel Core i5處理器
描述：一款適合日常使用的輕便筆記型電腦。
價格：$799.99

2. 產品：BlueWave 電競筆電
類別：電腦與筆記型電腦
品牌：BlueWave
型號：BW-GL200
保固：2年
評價：4.7
特性：15.6吋顯示器，16GB RAM，512GB SSD硬碟，NVIDIA GeForce RTX 3060顯示卡
描述：一款性能強勁的電競筆記型電腦，讓您沉浸其中。
價格：$1199.99

3. 產品：PowerLite 二合一電腦
類別：電腦與筆記型電腦
品牌：PowerLite
型號：PL-CV300
保固：1年
評價：4.3
特性：14吋觸控螢幕，8GB 記憶體，256GB SSD硬碟，360度螢幕轉軸
描述：一款多功能的轉換型筆記本，配備反應靈敏的觸控螢幕。
價格：$699.99

4. 產品：TechPro 桌機
類別：電腦與筆記型電腦
品牌：TechPro
型號：TP-DT500
保固：1年
評價：4.4
特性：Intel Core i7處理器，16GB 記憶體，256GB 硬碟，NVIDIA GeForce GTX 1660顯示卡
描述：一台適合工作與遊戲的強大桌上型電腦。
價格：$999.99

5. 產品：BlueWave Chromebook
類別：電腦與筆記型電腦
品牌：BlueWave
型號：BW-CB100
保固：1年
評價：4.1
特性：11.6吋顯示器，4GB 記憶體，256GB eMMC硬碟，Chrome OS
描述：一款輕巧且經濟實惠的Chromebook，適合完成日常任務。
價格：$249.99

步驟3：{delimiter} 如果訊息中包含上述列表中的產品， \
列出使用者訊息中可能的假設，例如筆記本X比筆記本Y大，或者筆記本Z有2年保固。

步驟4：{delimiter} 如果用戶有任何假設， \
根據您的產品資訊判斷該假設是否正確。

步驟5：{delimiter} 首先，如果適用，請禮貌地糾正客戶的錯誤假設。 \
只提及或參考上述五種產品資訊，因為這些是商店售賣的唯一五種產品。 \
以友善的語氣回答客戶。

使用以下格式：
步驟1：{delimiter} <步驟1的理由>
步驟2：{delimiter} <步驟2的理由>
步驟3：{delimiter} <步驟3的理由>
步驟4：{delimiter} <步驟4的理由>
回覆給使用者：{delimiter} <回覆給客戶的內容>

確保每一步都包含 {delimiter} 來進行分隔。
"""

使用者訊息測試（一）

In [15]:
user_message = f"""
你們的 BlueWave Chromebook 比 TechPro 桌機貴多少？"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

步驟1：#### 用戶在問關於特定產品的問題，不考慮產品類別。
步驟2：#### 該產品 BlueWave Chromebook 在產品列表中，價格為 $249.99。TechPro 桌機也在產品列表中，價格為 $999.99。
步驟3：#### 用戶沒有提供任何假設。
步驟4：#### 無需判斷假設是否正確。
回覆給使用者：#### BlueWave Chromebook 的價格是 $249.99，而 TechPro 桌機的價格是 $999.99。因此，BlueWave Chromebook 比 TechPro 桌機便宜了 $750。


使用者訊息測試（二）

In [4]:
user_message = f"""你們有賣電視嗎？"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

#### 步驟1：客戶詢問關於特定產品或多種產品的問題。不考慮產品類別。####

#### 步驟2：由於客戶沒有詢問特定產品，因此我們無法回答關於產品的問題。請問您需要關於哪種類型的產品的資訊？例如，電腦、手機、家電等等。####

回覆給客戶：#### 請問您需要關於哪種類型的產品的資訊？例如，電腦、手機、家電等等。####


## 回覆訊息的提取

由於我們要求 LLM 在進行思考過程和形成最終回應時，使用四個井字號作為分隔符，因此我們可以對訊息進行特殊處理，以便提取出最後的回覆。具體做法如下：

In [16]:
try:
    # 使用分隔符號來切割訊息，並且取出真的要回覆的內容
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    # 發現無法處理的訊息時的回覆訊息
    final_response = "很抱歉，我無法處理你這個問題，請您試著詢問其他問題。"
    
print(final_response)

BlueWave Chromebook 的價格是 $249.99，而 TechPro 桌機的價格是 $999.99。因此，BlueWave Chromebook 比 TechPro 桌機便宜了 $750。


## 額外測試

In [11]:
user_message = f"""我想買一台工作用的筆電，我經常會需要跟對面的客人做簡報， \
有沒有方便我這樣使用的機種推薦？"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

步驟1：####客戶在問關於特定產品或多種產品的問題。不考慮產品類別。####
步驟2：####辨認該產品是否在以下列表中。####

所有產品：
1. 產品：TechPro Ultrabook
類別：電腦與筆記型電腦
品牌：TechPro
型號：TP-UB100
保固：1年
評價：4.5
特性：13.3吋顯示器，8GB 記憶體，256GB SSD，Intel Core i5處理器
描述：一款適合日常使用的輕便筆記型電腦。
價格：$799.99

2. 產品：BlueWave Gaming Laptop
類別：電腦與筆記型電腦
品牌：BlueWave
型號：BW-GL200
保固：2年
評價：4.7
特性：15.6吋顯示器，16GB RAM，512GB SSD硬碟，NVIDIA GeForce RTX 3060顯示卡
描述：一款性能強勁的電競筆記型電腦，讓您沉浸其中。
價格：$1199.99

3. 產品：PowerLite Convertible
類別：電腦與筆記型電腦
品牌：PowerLite
型號：PL-CV300
保固：1年
評價：4.3
特性：14吋觸控螢幕，8GB 記憶體，256GB SSD硬碟，360度螢幕轉軸
描述：一款多功能的轉換型筆記本，配備反應靈敏的觸控螢幕。
價格：$699.99

4. 產品：TechPro Desktop
類別：電腦與筆記型電腦
品牌：TechPro
型號：TP-DT500
保固：1年
評價：4.4
特性：Intel Core i7處理器，16GB 記憶體，256GB 硬碟，NVIDIA GeForce GTX 1660顯示卡
描述：一台適合工作與遊戲的強大桌上型電腦。
價格：$999.99

5. 產品：BlueWave Chromebook
類別：電腦與筆記型電腦
品牌：BlueWave
型號：BW-CB100
保固：1年
評價：4.1
特性：11.6吋顯示器，4GB 記憶體，256GB eMMC硬碟，Chrome OS
描述：一款輕巧且經濟實惠的Chromebook，適合完成日常任務。
價格：$249.99

步驟3：####根據客戶的需求，我們建議選擇PowerLite Convertible或TechPro Ultrabook。這兩款筆記型電腦都輕巧便攜，並且配備了反應靈敏的觸控螢幕，方便您在簡報時進行操作。##